# Pre-trained 모델을 사용한 이미지 분류
## Using the SageMaker TensorFlow Serving Container

[SageMaker TensorFlow Serving Container](https://github.com/aws/sagemaker-tensorflow-serving-container)를 사용하면 사용자 지정 모델 로딩 또는 추론 코드 없이도 훈련된 TensorFlow 모델을 SageMaker Endpoint에 쉽게 배포할 수 있습니다. 이 예제에서는 [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk)를 사용하여 [TensorFlow Hub](https://www.tensorflow.org/hub/)에서 SageMaker Endpoint로 사전 학습된 모델을 배포한 다음, 추론 요청을 수행하는 방법을 보여줍니다.

## Setup

SageMaker Python SDK의 버전을 확인하고 추가적인 python packages를 설치합니다.

In [1]:
#!pip install -U --quiet "sagemaker>=1.14.2,<2"
!pip install -U --quiet opencv-python tensorflow-hub

SageMaker가 AWS 계정의 리소스에 액세스 할 수 있도록 노트북 환경에서 IAM 실행 역할을 가져옵니다.

In [2]:
import sagemaker   
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()
sagemaker_role = get_execution_role()

### 아래 두 값을 변경해주세요.

In [3]:
student_number = 'CHANGE TO YOUR STUDENT NUMBER'   # 여러분의 학번으로 변경해주세요. 예) '2021000000'
bucket = 'CHANGE TO YOUR S3 BUCKET NAME'           # 여러분의 S3 bucket 명으로 변경해주세요. 예) 'sagemaekr-000000000000'

## Download and prepare a model from TensorFlow Hub

TensorFlow Serving Container는 TensorFlow의 [SavedModel format](https://www.tensorflow.org/guide/saved_model) 형식으로 저장된 모든 모델에서 작동합니다. 훈련 작업의 결과이거나 다른 곳에서 훈련된 모델 모두 포함되며, 이 예제에서는 TensorFlow Hub의 ``MobileNet V2`` 라는 이미지 분류 모델의 사전 학습된 모델을 사용합니다. [TensorFlow Hub](https://tfhub.dev/) 모델은 사전 학습되었지만 제공하는 signature_def가 포함되어 있지 않으므로 모델을 TensorFlow 세션에 로드하고 입력 및 출력 레이어를 정의한 다음 저장된 모델로 내보내야합니다. 이 노트북의 ``sample_utils.py`` 모듈에는 이를 수행하는 함수가 있습니다.

In [4]:
from data import sample_utils

model_name = 'mobilenet_v2_140_224'
export_path = 'mobilenet'
model_path = sample_utils.tfhub_to_savedmodel(model_name, export_path)

print('SavedModel exported to {}'.format(model_path))

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.simple_save.


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.simple_save.


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


...

INFO:tensorflow:No assets to write.


INFO:tensorflow:No assets to write.


INFO:tensorflow:SavedModel written to: mobilenet/mobilenet_v2_140_224/00000001/saved_model.pb


INFO:tensorflow:SavedModel written to: mobilenet/mobilenet_v2_140_224/00000001/saved_model.pb


SavedModel exported to mobilenet/mobilenet_v2_140_224/00000001


In [5]:
!saved_model_cli show --all --dir {model_path}



MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['images'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 224, 224, 3)
        name: images:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['classes'] tensor_info:
        dtype: DT_INT32
        shape: (-1, 3)
        name: TopKV2:1
    outputs['probabilities'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 3)
        name: TopKV2:0
  Method name is: tensorflow/serving/predict


## Create a model archive file

SageMaker 모델은 `.tar.gz` 파일로 패키징해야합니다. 엔드포인트가 프로비저닝되면 아카이브의 파일이 추출되어 엔드포인트의 `/opt/ml/model/` 에 저장됩니다.

In [6]:
!tar -C "$PWD" -czf mobilenet.tar.gz mobilenet/

## Upload the model archive file to S3

SageMaker 모델을 생성하려면 먼저 S3에 업로드해야 하기에 SageMaker Python SDK를 사용하여 업로드합니다.

In [7]:
model_data = sagemaker_session.upload_data(path='mobilenet.tar.gz', bucket=bucket, key_prefix='img-classify/model')
print('model uploaded to: {}'.format(model_data))

model uploaded to: s3://cf-exam-s3-hosting/img-classify/model/mobilenet.tar.gz


## Create a SageMaker Model and Endpoint

이제 모델 아카이브가 S3에 있으므로 Python 코드를 사용하여 모델을 생성하고 엔드포인트에 배포할 수 있습니다.

In [8]:
import tensorflow as tf

print(tf.__version__) 
tf_framework_version = tf.__version__

1.15.5


In [9]:
from sagemaker.tensorflow.serving import Model

# Use an env argument to set the name of the default model.
# This is optional, but recommended when you deploy multiple models
# so that requests that don't include a model name are sent to a predictable model.
env = {'SAGEMAKER_TFS_DEFAULT_MODEL_NAME': 'mobilenet_v2_140_224'}

model = Model(model_data=model_data, 
              role=sagemaker_role, 
              framework_version=tf_framework_version, 
              env=env)

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [13]:
'''
%%time
from time import gmtime, strftime

endpoint_name=student_number+'-img-classify-'+strftime("%Y-%m-%d-%H-%M-%S", gmtime())

predictor = model.deploy(initial_instance_count=1, 
                         instance_type='ml.c5.xlarge', # ml.m4.xlarge
                         endpoint_name=endpoint_name) 
'''

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


-------------!CPU times: user 308 ms, sys: 23 ms, total: 331 ms
Wall time: 6min 31s


## Make predictions using the endpoint
 
이제 엔드포인트가 실행됐으며 추론을 위해 아래 sample 이미지를 사용할 것입니다.

<img src="./data/kitten.jpg" align="left" style="padding: 8px;">
<img src="./data/bee.jpg" style="padding: 8px;">

In [14]:
'''
# read the image files into a tensor (numpy array)
kitten_image = sample_utils.image_file_to_tensor('./data/kitten.jpg')

# get a prediction from the endpoint
# the image input is automatically converted to a JSON request.
# the JSON response from the endpoint is returned as a python dict
result = predictor.predict(kitten_image)

# show the raw result
print(result)
'''

{'predictions': [{'probabilities': [0.428937078, 0.317184091, 0.0627320185], 'classes': [283, 282, 286]}]}


### Add class labels and show formatted results

`sample_utils` 모듈을 통해 결과에 Imagenet 클래스 레이블을 추가하고 형식화된 출력을 보여줍니다.

In [15]:
'''
# add class labels to the predicted result
sample_utils.add_imagenet_labels(result)

# show the probabilities and labels for the top predictions
sample_utils.print_probabilities_and_labels(result)
'''

0.4289371 n02123159 tiger cat
0.3171841 n02123045 tabby, tabby cat
0.0627320 n02124075 Egyptian cat



## Stop / Close the Endpoint

본 예제를 모두 마무리한 후 아래 셀을 실행합니다. 다음 명령은 추론 단계에서 생성한 SageMaker에서 호스팅되고 있는 엔드포인트를 제거합니다. 엔드포인트를 삭제하지 않으면 계속 사용요금이 발생할 수 있습니다.

In [16]:
# sagemaker.Session().delete_endpoint(predictor.endpoint)

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
